<center> </center>

<center><font size=5 face="Helvetica" color=#76B900><b>
CUDA MODE: Lecture 1
</b></font></center>

<center><font face="Helvetica" size=3><b>Ang Chen</b></font></center>
<center><font face="Helvetica" size=3>September, 2024</font></center>

***

# Pytorch square

In [1]:
import torch
from torch.utils.cpp_extension import load_inline

In [2]:
a = torch.tensor([1., 2., 3.])

print(torch.square(a))
print(a ** 2)
print(a * a)

tensor([1., 4., 9.])
tensor([1., 4., 9.])
tensor([1., 4., 9.])


In [3]:
def time_pytorch_function(func, input):
    # CUDA IS ASYNC so can't use python time module
    start = torch.cuda.Event(enable_timing=True)
    end = torch.cuda.Event(enable_timing=True)

    # Warmup
    for _ in range(5):
        func(input)

    start.record()
    func(input)
    end.record()
    torch.cuda.synchronize()
    return start.elapsed_time(end)

In [4]:
b = torch.randn(10000, 10000).cuda()

def square_2(a):
    return a * a

def square_3(a):
    return a ** 2

In [5]:
time_pytorch_function(torch.square, b)
time_pytorch_function(square_2, b)
time_pytorch_function(square_3, b)

print("=============")
print("Profiling torch.square")
print("=============")

Profiling torch.square


In [6]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    torch.square(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

print("=============")
print("Profiling a * a")
print("=============")

---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
         aten::square        26.42%     214.000us       100.00%     810.000us     810.000us     587.000us        13.80%       4.254ms       4.254ms             1  
            aten::pow        73.33%     594.000us        73.58%     596.000us     596.000us       3.665ms        86.15%       3.667ms       3.667ms             1  
    aten::result_type         0.12%       1.000us         0.12%       1.000us       1.000us       1.000us         0.02%       1.000us       1.000us             1  
             at

In [7]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_2(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

print("=============")
print("Profiling a ** 2")
print("=============")

-------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
         Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
    aten::mul       100.00%      49.000us       100.00%      49.000us      49.000us       3.528ms       100.00%       3.528ms       3.528ms             1  
-------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
Self CPU time total: 49.000us
Self CUDA time total: 3.528ms

Profiling a ** 2


In [8]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    square_3(b)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                 Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
---------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
            aten::pow        97.33%      73.000us       100.00%      75.000us      75.000us       3.118ms        99.94%       3.120ms       3.120ms             1  
    aten::result_type         1.33%       1.000us         1.33%       1.000us       1.000us       1.000us         0.03%       1.000us       1.000us             1  
             aten::to         1.33%       1.000us         1.33%       1.000us       1.000us       1.000us         0.03%       1.000us       1.000us             1  
---------------

# Hello load inline

In [9]:
cpp_source = """
std::string hello_world() {
    return "Hello World!";
}
"""

In [10]:
my_module = load_inline(
    name='my_module',
    cpp_sources=[cpp_source],
    functions=['hello_world'],
    verbose=True,
    build_directory='./tmp'
)

my_module.hello_world()

Emitting ninja build file ./tmp\build.ninja...
Building extension module my_module...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
Loading extension module my_module...


'Hello World!'